## 모듈 불러오기

In [1]:
import pandas as pd

import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

from collections import Counter
from collections import defaultdict

import re

from tqdm import tqdm


### 5. paperswithcode dataset (크롤링)

In [36]:

'''
1. see all categories (click) 
    기존 : (6,3) : general 제외, graph 예외처리 
    변경 : (4,3)클릭 위치 변경을 통한 예외를 없애줌 
<기존>
body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(6) > a    
    
<변경>
body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(4) > div > h4 > a
body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(7) > div > h4 > a


2. method (고정값)
body > div.container.content.content-buffer > div > div.title.browse-methods-title > div:nth-child(2) > div > h1


3. see all models (click)
    (3,3)

body > div.container.content.content-buffer > div > div.infinite-container.featured-methods > div:nth-child(3) > a

4. categories (고정값)
body > div.container.content.content-buffer > div.mobile-width > div.artefact-header > h1

5. models
    (1,1) : category에 해당하는 모델이 없는 경우(예외 처리)
    #methodsTable > tbody > tr:nth-child(1) > td:nth-child(1) > div.method-image > a
    #methodsTable > tbody > tr:nth-child(2) > td:nth-child(1) > div.method-image > a
'''

'\n1. see all categories (click) \n    기존 : (6,3) : general 제외, graph 예외처리 \n    변경 : (4,3)클릭 위치 변경을 통한 예외를 없애줌 \n<기존>\nbody > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(6) > a    \n    \n<변경>\nbody > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(4) > div > h4 > a\nbody > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child(7) > div > h4 > a\n\n\n2. method (고정값)\nbody > div.container.content.content-buffer > div > div.title.browse-methods-title > div:nth-child(2) > div > h1\n\n\n3. see all models (click)\n    (3,3)\n\nbody > div.container.content.content-buffer > div > div.infinite-container.featured-methods > div:nth-child(3) > a\n\n4. categories (고정값)\nbody > div.container.content.content-buffer > div.mobile-width > div.artefact-header > h1\n\n5. models\n    (1,1) : category에 해당하는 모델이 없는 경우(예외 처리

- Main Code (주석 해제 후 사용)

In [ ]:
'''
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup 
from selenium.common.exceptions import NoSuchElementException

driver = webdriver.Chrome()
driver.get('https://paperswithcode.com/methods')
data_list = []  
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

tmp1=4
while True: # 1,2번   
    try:
        click1_selector = f'body > div.container > div.container.content.content-buffer > div.infinite-container.featured-methods > div:nth-child({tmp1}) > div > h4 > a'
        temp_categories = driver.find_element(By.CSS_SELECTOR,click1_selector) #
    except NoSuchElementException:       
            break
    driver.execute_script("arguments[0].click();", temp_categories)
    tmp1 +=3
    time.sleep(1.5)
    
    # 클릭으로 인한 페이지 업데이트
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    method_selector = 'body > div.container.content.content-buffer > div > div.title.browse-methods-title > div:nth-child(2) > div > h1'
    method_name = soup.select(method_selector)
    if not method_name:
        break
    method = method_name[0].text.strip().replace('\n','')
    print('method: ',method)
    
    tmp2 = 3
    while True: # 3,4번
        try:
            click2_selector = f'body > div.container.content.content-buffer > div > div.infinite-container.featured-methods > div:nth-child({tmp2}) > a'
            temp_model = driver.find_element(By.CSS_SELECTOR,click2_selector)
        except NoSuchElementException: 
            break
        driver.execute_script("arguments[0].click();", temp_model)
        tmp2 += 3
        time.sleep(1.5)
        
        # 페이지 업데이트
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        #here
        category_selector = 'body > div.container.content.content-buffer > div.mobile-width > div.artefact-header > h1'
        category_name = soup.select(category_selector)
        if not category_name:
            break
        category = category_name[0].text.split("\xa0")[0].strip()
        print('category: ',category)

        model_list = []
        tmp3 = 1
        while True: # 5번
            model_selector = f'#methodsTable > tbody > tr:nth-child({tmp3}) > td:nth-child(1) > div.method-image > a'
            model_name = soup.select(model_selector) 
            if not model_name:
                break
            model = model_name[0].text.strip().replace('\n', '')
            model_list.append(model)
            print("model: ",model)
            tmp3 += 1
            
        data_list.append([method,category,', '.join(model_list)])
        
        driver.back()
    driver.back()

# 데이터를 CSV 파일로 저장
with open('cw_paperswithcode.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['method', 'category', 'model'])  # 헤더 추가
    csv_writer.writerows(data_list)  # 데이터 추가
'''

- csv 구조 변경 (행열 변환)

In [ ]:
import csv

# 입력 파일과 출력 파일 이름\
input_filename = r'C:\Users\ASUS\Desktop\coding\FastCampus\eda\project\vscode\cw_paperswithcode.csv'
output_filename = r'C:\Users\ASUS\Desktop\coding\FastCampus\eda\project\vscode\paperswithcode.csv'
# CSV 파일을 읽고 데이터를 리스트로 저장
data = []
with open(input_filename, 'r', newline='', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        data.append(row)

# 데이터를 풀어서 저장할 새로운 리스트 생성
unrolled_data = []
for row in data:
    method = row[0]
    category = row[1]
    models = row[2].split(', ')  # ,를 기준으로 model들을 분리
    for model in models:
        unrolled_data.append([method, category, model])

# 새로운 CSV 파일로 데이터를 쓰기
with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    for row in unrolled_data:
        csv_writer.writerow(row)